# Домашнее задание: Учимся работе с данными - от загрузки до моделирования.

## 1. Скачайте датасет

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
import seaborn as sb
import matplotlib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('dataset.csv')
df.head()

## 2. Посмотрите на базовые статистики датасета: средние, медианы, и т.д.

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.iloc[:, 1:-1] 
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

## 3. Постройте гистограммы\распределения признаков, при этом используйте целевую переменную, чтобы сгруппировать и раскрасить гистограммы.

In [ ]:
fig,ax1 = plt.subplots()
sb.histplot(df['diagnosis'], fill=False, ax=ax1,  color='skyblue' )
ax1.set_title(r'Распределение признака diagnosis')
plt.show()

In [ ]:
df['diagnosis'].value_counts(normalize=True)

In [ ]:
limit = 150
ax = sb.histplot(x='perimeter_mean', data=df[df['perimeter_mean'] <= limit],
                  bins=10, binrange=(df['perimeter_mean'].min(), limit), color='green')
sb.histplot(x='perimeter_mean', data=df[df['perimeter_mean'] > limit],
             bins=3, binrange=(limit, df['perimeter_mean'].max()), color='red', ax=ax)
plt.axvline(limit, color='black', linestyle='--')
plt.show()

In [ ]:
df.hist(figsize=(20, 15));

In [ ]:
sb.pairplot(df, hue='diagnosis');

## 4. Постройте heatmap для матрицы корреляций, есть ли признаки, которые сильно скоррелированы? Какие это признаки?

In [ ]:
plt.figure(figsize=(20, 18))
sb.heatmap(df.corr(numeric_only=True), annot=True, cbar=False, cmap='viridis', fmt=".2f")
plt.title('Heatmap для матрицы корреляции')
plt.show()

Наиболее скоррелированные признаки отмечены желтым цветом. Очевидно, что наибольшая корреляция между признаками: площадь, радиус, периметр

## 5. Постройте для сильно скоррелированных признаков попарные scatterplot-ы, действительно ли наблюдается линейная зависимость?

In [ ]:
df_corr = df.corr(numeric_only=True)
highly_corr = df_corr[df_corr.abs() > 0.9]
highly_corr = highly_corr.stack().reset_index()
highly_corr = highly_corr[highly_corr['level_0'] < highly_corr['level_1']]
#сильно скоррелированные признаки 
print(highly_corr)
for index, row in highly_corr.iterrows():
    plt.figure(figsize=(6, 4))
    sb.scatterplot(data=df, x=row['level_0'], y=row['level_1'])
    plt.title(f"Корреляция признака {row['level_0']} от {row['level_1']}")
    plt.show()

В данных наблюдается линейная зависимость

## 6. Используя boxplots и группировку по целевой переменной, попробуйте предположить, по каким признакам наиболее удобно было бы отделить злокачественные новообразования от доброкачественных.

In [ ]:
features = [col for col in df.columns if col != 'diagnosis']
for feature in features:
    plt.figure(figsize=(6, 4))
    sb.boxplot(x='diagnosis', y=feature, data=df)
    plt.title(f'Boxplot признака {feature} сгруппированный по полю diagnosis')
    plt.show()

Проанализировав графики, можно предположить, что наиболее показательными признаками для отделения видов диагноза являются: rasius_mean,  perimeter_mean, area_mean, compactness_mean, concavity_mean, concave points_mean,  concave points_worst, concavity_worst

## Часть 2. Моделирование при помощи kNN
### Приведите все непрерывные переменные к одному масштабу при помощи стандартизации. Кратко поясните, почему стандартизация здесь нужна.

In [ ]:
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
print(non_numeric_cols)


In [ ]:
df['diagnosis'] = df['diagnosis'].apply(lambda x: 1 if x=='M' else 0)
df.head()

### Разбейте данные на train-test, отложив 30% выборки для тестирования.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['diagnosis'], axis=1), df['diagnosis'], test_size=0.30, random_state=42, stratify=df['diagnosis']
)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
y_test.value_counts(normalize=True)

#### Необходма стандартизация данных, так как присутствует разный порядок значений

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Постройте модель kNN "из коробки" без настройки параметров. Оцените метрики качества бинарной классификации (accuracy, precision, recall, f1_score), постройте ROC-кривую и посчитайте площадь под ней.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)

y_train_pred = knn.predict(X_train_scaled)
y_test_pred = knn.predict(X_test_scaled)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Train metrics:')
print(f'Accuracy: {accuracy_score(y_train, y_train_pred)}')
print(f'Precision: {precision_score(y_train, y_train_pred)}')
print(f'Recall: {recall_score(y_train, y_train_pred)}')
print(f'F1-score: {f1_score(y_train, y_train_pred)}')

print('Test metrics:')
print(f'Accuracy: {accuracy_score(y_test, y_test_pred)}')
print(f'Precision: {precision_score(y_test, y_test_pred)}')
print(f'Recall: {recall_score(y_test, y_test_pred)}')
print(f'F1-score: {f1_score(y_test, y_test_pred)}')

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score


y_test_prob = knn.predict_proba(X_test_scaled)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_test_prob)  # вычисление значений для ROC кривой
roc_auc = roc_auc_score(y_test, y_test_prob)  # вычисление площади под ROC кривой

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkgreen', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

Можно оцени модель как вполне точную

## Теперь проведите настройку параметра числа соседей на кросс-валидации. Снова оцените результаты, стало ли лучше?

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
        'n_neighbors': range(1, 50),
        'p': range(1, 4)
}

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='f1', verbose=1)
grid_search.fit(X_train_scaled, y_train)

In [ ]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_}")

In [ ]:
y_train_pred = grid_search.predict(X_train_scaled)
y_test_pred = grid_search.predict(X_test_scaled)

print('Train metrics:')
print(f'Accuracy: {accuracy_score(y_train, y_train_pred)}')
print(f'Precision: {precision_score(y_train, y_train_pred)}')
print(f'Recall: {recall_score(y_train, y_train_pred)}')
print(f'F1-score: {f1_score(y_train, y_train_pred)}')

print('Test metrics:')
print(f'Accuracy: {accuracy_score(y_test, y_test_pred)}')
print(f'Precision: {precision_score(y_test, y_test_pred)}')
print(f'Recall: {recall_score(y_test, y_test_pred)}')
print(f'F1-score: {f1_score(y_test, y_test_pred)}')

y_test_prob = grid_search.predict_proba(X_test_scaled)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_test_prob)  # вычисление значений для ROC кривой
roc_auc = roc_auc_score(y_test, y_test_prob)  # вычисление площади под ROC кривой

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkgreen', lw=lw, label=f'ROC curve (area = {roc_auc:0.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

Настройка параметра числа соседей на кросс-валидации привела к улучшению метрик